In [25]:
# coding:utf-8
# Textual analysis through the open web
from __future__ import print_function
import io
import os
import re
import bs4
import sys
import json
import time
import urllib
import pprint
import random
import requests
import wikipedia

In [2]:
def read_json(path):
    data = ''
    with io.open(path, 'r', encoding='utf-8') as f:
        data = json.loads(f.read())
        print(__message(u'Loaded {}'.format(path)))
    return data

def scrape_wiki(title):
    try:
        searchstring = title
        summary = wikipedia.summary(searchstring)
        print(__success(searchstring))
    except wikipedia.DisambiguationError:
        try:
            searchstring = u'{} (video game)'.format(title).replace(u' ', u'_')
            summary = wikipedia.page(searchstring, auto_suggest=False).summary
            print(__success(searchstring))
        except wikipedia.DisambiguationError:
            searchstring = u'{} (Unix video game)'.format(title).replace(u' ', u'_')
            summary = wikipedia.page(searchstring, auto_suggest=False).summary
            print(__success(searchstring))
        except wikipedia.PageError:
            summary = 'No summary found on Wikipedia.'
            print(__warning(u'Wikipedia cannot find "{}"'.format(searchstring)))
    except wikipedia.PageError:
        try:
            summary = wikipedia.page(searchstring, auto_suggest=False).summary
            print(__success(searchstring))
        except wikipedia.PageError:
            summary = 'No summary found on Wikipedia.'
            print(__warning(u'Search term "{}" returned nothing'.format(searchstring)))
    return summary

def scrape_duckduckgo(keywords, developer=""):
    searchstring = u'"{}" {} {}'.format(keywords, developer, u'interview game')
    response = requests.get(u'http://duckduckgo.com/html/?q={}'.format(
                urllib.quote(searchstring.encode('utf-8'))),
                            timeout=(9.1, 12.1)
             )
    print(__message(u'DDG: {}'.format(searchstring)))
    soup = bs4.BeautifulSoup(response.text)
    links = []
    for node in soup.select('div.web-result'):
        if 'web-result-sponsored' in node['class']:
            continue
        try:
            links.append(node.select('a.large')[0].get('href'))
        except Exception as e:
            print(__failure(e))
            pass
    if links:
        print(__success(u'DDG: {}'.format(searchstring)))
        print('\t\t\n'.join(links))
    return links

def scrape(url):
    try:
        response = requests.get(url, timeout=(10, 15))
    except Exception as e:
        print(__failure(e))
        return ''
    
    return response.text
    
def save_json(path, data):
    with io.open(path, 'w', encoding='utf-8') as f:
        try:
            output = json.dumps(data, indent=2, ensure_ascii=False)
            f.write(output)
        except UnicodeEncodeError:
            f.write(output.encode('utf-8'))
    print(__message(u'Written to {}'.format(path)))
        
def __success(text):
    return u'  (SUCC) {}'.format(text).encode('utf-8')
    
def __failure(text):
    return u'!!FAIL!! {}'.format(text).encode('utf-8')
    
def __warning(text):
    return u'??WARN?? {}'.format(text).encode('utf-8')
    
def __message(text):
    return u'   |MSG| {}'.format(text).encode('utf-8')

In [ ]:
# Scrape for links
game_meta = read_json(os.path.join(os.getcwd(), 'data', 'game-sources.json'))

shuffled_game_meta = game_meta.items()
random.shuffle(shuffled_game_meta)
for game, meta in shuffled_game_meta:
    game_meta[game]['Links'] += scrape_duckduckgo(game, game_meta[game]['Developer'])
    game_meta[game]['Links'] = list(set(game_meta[game]['Links']))
    save_json(os.path.join(os.getcwd(), 'data', 'game-sources.json'), game_meta)
    time.sleep(1)

In [ ]:
# Load URLs in interviews
game_meta = read_json(os.path.join(os.getcwd(), 'data', 'game-sources.json'))
cached = read_json(os.path.join(os.getcwd(), 'data', 'interviews.json'))

output = cached
for game, meta in game_meta.items():
    if game not in output:
        output[game] = {}
    print(__message(game))
    for url in meta['Links']:
        if url in output[game]:
            continue
        print(__message(url))
        html = scrape(url)
        if 'interview' in url.lower() or 'interview' in html.lower():
            soup = bs4.BeautifulSoup(html)
            content = soup.select('div > p') + soup.select('body > p')
            data = [c.string.strip() for c in content if c.string and c.string.strip()]
            output[game][url] = data
            save_json(os.path.join(os.getcwd(), 'data', 'interviews.json'), output)
        else:
            output[game][url] = []
        save_json(os.path.join(os.getcwd(), 'data', 'interviews.json'), output)

In [27]:
# Locate mentions of games
game_meta = read_json(os.path.join(os.getcwd(), 'data', 'game-sources.json'))
game_interviews = read_json(os.path.join(os.getcwd(), 'data', 'interviews.json'))

# Create a look up table for games
game_LUT = {}
for game, meta in game_meta.items():
    game_LUT[game] = game
    if 'AKA' in meta:
        for aka in meta['AKA']:
            game_LUT[aka] = game
            
            
pprint.pprint(game_LUT, indent=2)

   |MSG| Loaded /Users/spaxe/dev/roguelike-universe/data/game-sources.json
   |MSG| Loaded /Users/spaxe/dev/roguelike-universe/data/interviews.json
{ u'100 Rogues': u'100 Rogues',
  u'99 Levels to Hell': u'99 Levels to Hell',
  u'ADOM': u'Ancient Domains of Mystery',
  u'ADVENT': u'Adventure',
  u'Advanced Dungeons & Dragons: Cloudy Mountain': u'Advanced Dungeons & Dragons: Cloudy Mountain',
  u'Adventure': u'Adventure',
  u'Alphaman': u'Alphaman',
  u'Ancient Domains of Mystery': u'Ancient Domains of Mystery',
  u'Angband': u'Angband',
  u'Azure Dreams': u'Azure Dreams',
  u'Baroque': u'Baroque',
  u'Beneath Apple Manor': u'Beneath Apple Manor',
  u'Blue Rescue Team': u'Pok\xe9mon Mystery Dungeon',
  u'Brogue': u'Brogue',
  u'Castle of the Winds': u'Castle of the Winds',
  u'Chocobo no Fushigina Dungeon': u'Chocobo no Fushigina Dungeon',
  u"Chocobo's Dungeon 2": u"Chocobo's Dungeon 2",
  u"Chocobo's Mysterious Dungeon": u'Chocobo no Fushigina Dungeon',
  u'Colossal Cave': u'Adventure

In [ ]:

# Look through the interview articles
for game, interviews in game_interviews.items():
    print(__message(game))
    for url, interview in interviews.items():
        for line in interview:
            titles_found = set()
            for title in game_LUT.keys():
                try:
                    i = line.index(title)
                    if (i-1 >= 0 and line[i-1].isalpha()) or \
                       (i+len(title) < len(line) and line[i+len(title)].isalpha()):
                        continue # Not a whole word
                    elif game_LUT[title] not in titles_found:
                        titles_found.add(game_LUT[title])
                except ValueError:
                    continue
            if titles_found:
                for title in titles_found:
                    line = line.replace(title, '***{}***'.format(title))
                print(line)